In [ ]:
import requests
import json

# Danish Superliga id: 271
# Scottish Prem id: 501

API_TOKEN = json.load(open('/home/morten/Develop/secrets.json', 'r'))['sportmonks']['API_TOKEN']
Danish_SL = 271
Scottish_Prem = 501

Get all available Seasons

In [ ]:
request = f"https://soccer.sportmonks.com/api/v2.0/seasons?api_token={API_TOKEN}&league={Danish_SL}"
# request = f"https://soccer.sportmonks.com/api/v2.0/fixtures/date/2020-08-02?api_token={API_TOKEN}&leagues=271"
r = requests.get(request)
print(r)
data = r.json()
# print(data)
for x in data["data"]:
    print(x)

In [ ]:
D_first_season = 17328
request = f"https://soccer.sportmonks.com/api/v2.0/rounds/season/{D_first_season}?api_token={API_TOKEN}&include=fixtures"

r = requests.get(request)
print(r)
data = r.json()
game_list = []
for gameday in data["data"]:
    for game in gameday["fixtures"]["data"]:
        print(
            f"{gameday['name']}, {game['id']}, {game['localteam_id']}, {game['visitorteam_id']}"
        )
        game_list.append(game["id"])

In [ ]:
D_first_match = 16773975

request = f"https://soccer.sportmonks.com/api/v2.0/fixtures/{D_first_match}?api_token={API_TOKEN}&include=lineup,substitutions,events"

r = requests.get(request)

print(r)

data = r.json()

print(data)

In [ ]:
for x in data["data"]["lineup"]["data"]:
    print(str(x["team_id"]) + " " + str(x["player_id"]) + " " + x["player_name"])

print("------------------------------------------")

for x in data["data"]["substitutions"]["data"]:
    print(
        "In : "
        + str(x["team_id"])
        + " "
        + str(x["player_in_id"])
        + " "
        + x["player_in_name"]
        + " "
        + str(x["minute"])
    )
    print(
        "Out: "
        + str(x["team_id"])
        + " "
        + str(x["player_out_id"])
        + " "
        + x["player_out_name"]
        + " "
        + str(x["minute"])
    )

print("------------------------------------------")

for x in data["data"]["events"]["data"]:
    if x["type"] == "yellowcard":
        print(
            "Yellow: "
            + str(x["player_id"])
            + " "
            + x["player_name"]
            + " "
            + str(x["minute"])
        )

In [ ]:
request = f"https://soccer.sportmonks.com/api/v2.0/fixtures/{D_first_match}?api_token={API_TOKEN}&include=corners"

r = requests.get(request)

print(r)

data = r.json()

print(data)

In [ ]:
for idx, x in enumerate(data["data"]["corners"]["data"]):
    print(
        "minute: "
        + str(x["minute"])
        + ", corner number: "
        + str(idx + 1)
        + ", team: "
        + str(x["team_id"])
    )

In [ ]:
request = f"https://soccer.sportmonks.com/api/v2.0/fixtures/{D_first_match}?api_token={API_TOKEN}&include=lineup,substitutions,events,corners"

r = requests.get(request)

print(r)

data = r.json()

# print(data)
print(json.dumps(data, indent=2))

if data["data"]["corners"]["data"]:
    print("corners is empty: skip")
if data["data"]["lineup"]["data"]:
    print("lineup is empty: skip")

In [ ]:
localTeam = data["data"]["localteam_id"]
visitorTeam = data["data"]["visitorteam_id"]
local_corners = []
visitor_corners = []
players_local = []
players_away = []

for x in data["data"]["corners"]["data"]:
    if x["comment"][0] == "R":
        continue  # skip "Race to Nth Corner events"
    if x["team_id"] == localTeam:
        local_corners.append(x["minute"])  # add minute of corner to list of corners
    if x["team_id"] == visitorTeam:
        visitor_corners.append(x["minute"])

for x in data["data"]["lineup"]["data"]:
    if x["team_id"] == localTeam:
        players_local.append(
            [x["player_id"], 0, x["stats"]["other"]["minutes_played"], False]
        )
    if x["team_id"] == visitorTeam:
        players_away.append(
            [x["player_id"], 0, x["stats"]["other"]["minutes_played"], False]
        )

for x in data["data"]["substitutions"]["data"]:
    if x["team_id"] == str(localTeam):
        players_local.append([x["player_in_id"], x["minute"], 90, True])
    if x["team_id"] == str(visitorTeam):
        players_away.append([x["player_in_id"], x["minute"], 90, True])

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect("cI.db")

cur = con.cursor()

# for each match
localTeam = data["data"]["localteam_id"]
visitorTeam = data["data"]["visitorteam_id"]
# date to string
date = data["data"]["time"]["starting_at"]["date"]
date = date.split("-")
date = date[0] + date[1] + date[2]
print(date)
# add corners!
for x in data["data"]["corners"]["data"]:
    if x["comment"][0] == "R":
        continue  # skip "Race to Nth Corner events"
    team_id = x["team_id"]
    corner_min = x["minute"]
    cur.execute(
        f"INSERT INTO game (game_id, corner_min, team_id, date) VALUES ({D_first_match},{corner_min},{team_id},{date})"
    )


for x in data["data"]["lineup"]["data"]:
    player_id = x["player_id"]
    home = 0 if x["team_id"] == str(localTeam) else 1
    min_on = 0
    min_off = x["stats"]["other"]["minutes_played"]
    cur.execute(
        f"INSERT INTO lineup (game_id, player_id, home, min_on, min_off, team_id) VALUES ({D_first_match},{player_id},{home},{min_on},{min_off},{x['team_id']})"
    )

for x in data["data"]["substitutions"]["data"]:
    player_id = x["player_in_id"]
    home = 0 if x["team_id"] == localTeam else 1
    min_on = x["minute"]
    min_off = 90
    cur.execute(
        f"INSERT INTO lineup (game_id, player_id, home, min_on, min_off, team_id) VALUES ({D_first_match},{player_id},{home},{min_on},{min_off},{x['team_id']})"
    )

con.commit()